Task 3: Predict Patient Health Categories 

1. For the first step of preprocessing the dataset, I imported beneficial libraries for this task and also assigned df as the variable that holds the dataset.

In [ ]:
import pandas as pd   # This import will be used for data manipulation
import numpy as np   # This import will be used for working with arrays
import seaborn as sns   # This import will be used for data visualisation and dataset loading 
import matplotlib.pyplot as plt   # This import will be used for plotting
from sklearn.model_selection import train_test_split   # This import will be used for splittig data into train and test sets 
from sklearn.preprocessing import StandardScaler   # This import will be used for Scaling Features  

df1 = pd.read_csv("test_data.csv")   # Loads the dataset into the variable df1
df = df1.drop("vitals_blood_pressure", axis = 1)   # Due to blood pressure being displayed as systolic/diastolic this would alter the 
print("Dataset preview:\n", df.head())   # Displays the first few rows of th dataset

In [ ]:
# This will allow to view the basic information about the dataset
print(df.info())   # Output: this will show the column names, data types, non-null counts and so on 

# We need to now also view the statistical summary of numrical columns 
print(df.describe())   # This wil output thingd such as the mean, std, min, 25%, 50%, 75%, max for numerical columns 

In [ ]:
# As the dataset contains string variables, I will encode sleep quality to integers
df['sleep_quality'] = df['sleep_quality'].map({'poor':0, 'fair':1, 'good':2, 'excellent':3})   # This will map 'sleep_quality' to 0, 1, 2 and 3 for poor, fair, good and excellent 

print(df)

In [ ]:

def seperate_string(val):   # Function to seperate the string and remove the '|' from the data points 
    try:
        return np.mean([float(x) for x in val.split('|')])
    except Exception as e:
        return np.nan

df['vitals_heart_rate'] = df['vitals_heart_rate'].apply(seperate_string)   # Replaces multiple data in the data point with the mean vital heart rate
df['vitals_temperature'] = df['vitals_temperature'].apply(seperate_string)   # Replaces multiple data in the data point with the mean vital temperature

print("\nData with mean:")
print(df.head())